In [1]:
import sys
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv('./data/train.csv',encoding='big5')
print(data)

              日期  測站          測項     0     1     2     3     4     5     6  \
0       2014/1/1  豐原    AMB_TEMP    14    14    14    13    12    12    12   
1       2014/1/1  豐原         CH4   1.8   1.8   1.8   1.8   1.8   1.8   1.8   
2       2014/1/1  豐原          CO  0.51  0.41  0.39  0.37  0.35   0.3  0.37   
3       2014/1/1  豐原        NMHC   0.2  0.15  0.13  0.12  0.11  0.06   0.1   
4       2014/1/1  豐原          NO   0.9   0.6   0.5   1.7   1.8   1.5   1.9   
...          ...  ..         ...   ...   ...   ...   ...   ...   ...   ...   
4315  2014/12/20  豐原         THC   1.8   1.8   1.8   1.8   1.8   1.7   1.7   
4316  2014/12/20  豐原       WD_HR    46    13    61    44    55    68    66   
4317  2014/12/20  豐原  WIND_DIREC    36    55    72   327    74    52    59   
4318  2014/12/20  豐原  WIND_SPEED   1.9   2.4   1.9   2.8   2.3   1.9   2.1   
4319  2014/12/20  豐原       WS_HR   0.7   0.8   1.8     1   1.9   1.7   2.1   

      ...    14    15    16    17    18    19    20    21    22

In [5]:
# 我们只需要第三列以后的data
data = data.iloc[:,3:]
# 将NR的数据全部置为0
data[data == 'NR'] = 0
# 将dataframe幻化成numpy
raw_data = data.to_numpy()

#查看raw_data
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


In [6]:
# 现在要开始处理Training data
# 由于提供的数据是每个月前20天(每天24小时)的资料(共18种类型)，因此每个月的这20*24=480个小时的时间是连续的，每10个小时可以作为一笔data，480小时共471笔data
# 由于不同月份之间的时间是不连续的，因此先把所有的资料按照月份分开
month_data = {}
for month in range(12):
  # 每个月共20*24=480份数据，共18种空气成分，每一种成分每月都有480份数据，因此初始化一个18*480的array
  temp_data = np.empty([18,480])
  for day in range(20):
    # temp_data中加入第day天的数据，由于每天都有24份数据共24列,故列的范围是24*day~24*(day+1)；选择加入temp_data的是第20×month+day这一天的数据，由于每天有18种大气成分的数据共18行，因此行的范围是18*(20*month+day)~18*(20*month+day+1)
    temp_data[:, 24 * day : 24 * (day + 1)] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1)]
  # 把这个月的data放进month_data[month]里去
  month_data[month]=temp_data
print(month_data[0])

[[14.   14.   14.   ... 14.   13.   13.  ]
 [ 1.8   1.8   1.8  ...  1.8   1.8   1.8 ]
 [ 0.51  0.41  0.39 ...  0.34  0.41  0.43]
 ...
 [35.   79.    2.4  ... 48.   63.   53.  ]
 [ 1.4   1.8   1.   ...  1.1   1.9   1.9 ]
 [ 0.5   0.9   0.6  ...  1.2   1.2   1.3 ]]


In [7]:
# 这个时候我们就要在12段连续的时间里每10个小时取出一笔Training data
# 每个月连续时间有20*24=480h，每10h作为一笔data，共可以分为471笔data；一共12个月，因此共12*471笔data
# 每一笔data，前9h为input，最后1h的PM2.5为output，因此input是一个18*9的矩阵，如果把它摊平就是一个18*9的feature vector

# x是input，共12*471笔data，每个input是一个18*9的feature vector
x = np.empty([12 * 471, 18 * 9], dtype = float)
# y是output，共12*471笔data，每个output是第10个小时的第9种大气成分PM2.5，因此只有1维
y = np.empty([12 * 471, 1], dtype = float)

# 取出input和output
for month in range(12):
  # 每个月份共471笔data，依次遍历即可
  for i in range(471):
    # 用一个18行(18种物质),10列(10个小时的数据，包括input和output)的temp_data来存放每一笔Training data
    temp_data = np.empty([18, 10])
    # 第i笔data的范围是i~i+9，共10列，注意这里i+10是开区间，实际只取到了i+9
    temp_data = month_data[month][:, i : i + 10]
    # 将temp_data前9列的作为input的数据(18行9列)用reshape平摊到一个18*9的行向量上
    x[471 * month + i, :] = temp_data[:, 0 : 9].reshape(1,-1)
    # temp_data的第10列作为output，只有第10行的PM2.5值是有用的
    y[471 * month + i] = temp_data[9, 9]

print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [8]:
# 由于要尝试不同normalize的效果，这里先用x_new保留一份原始的x,先用copy()函数复制一份x
x_new = x.copy()

# 直接调用numpy的mean和std计算期望和标准差，axis=0表示对列进行计算
mean_x = np.mean(x, axis = 0)
std_x = np.std(x, axis = 0)

# 对每一个feature都进行normalize归一化处理
for i in range(len(x)):
  for j in range(len(x[0])):
    if std_x[j] != 0:
      x[i][j] = (x[i][j] - mean_x[j])/std_x[j]

# 查看normalize后的input x
print(x)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.38950555 -0.10906991
   0.07797893]]
